# Demo

In [ ]:
# load all required packages at the start
import numpy as np
import matplotlib.pyplot as plt

from chebpy import chebfun

The function ``chebfun`` behaves in essentially the same way as its MATLAB counterpart. A good way to begin is to type:

In [ ]:
x = chebfun('x', [0, 10])
x #TODO: why doesn't repr or str work with print?

What's happened here is we've instantiated a numerical representation of the identity function on the interval `[0,10]` and assigned this to a computer variable `x`. This particular representation has length 2, meaning that it is a degree one polynomial defined via two degrees of freedom (as you would expect of a linear function).

An intuitive set of composition-like operations can now be performed. For instance here is the specification of a function `f` that oscillates with two modes:

In [ ]:
f = np.sin(x) + np.sin(5*x)
f #TODO: why doesn't repr or str work with print?

The zeros of f can be computed via `roots`, which behind the scenes is implemented via a recursive subdivision algorithm in which a number of Colleague Matrix eigenvalue sub-problems are solved:

In [ ]:
r = f.roots()
r #TODO: why doesn't repr or str work with print?

By default ChebPy computations are accurate to machine precision, or approximately fifteen digits in double-precision arithmetic (see also the `UserPrefs` interface [here](./implementation.ipynb)).

We can verify this for the computed roots of `f` by typing:

In [ ]:
f(r)

The function and its roots can be plotted together as follows:


In [ ]:
ax = f.plot()
ax.plot(r, f(r), 'or')
plt.show()

Calculus operations are natively possible with Chebfun objects. For example here is the derivative and indefinite integral of `f`:

In [ ]:
Df = f.diff()
If = f.cumsum()
f.plot()
Df.plot()
If.plot()
plt.show()

One can verify analytically that the exact value of the definite integral here is `1.2 - cos(10) - 0.2cos(50)`.

This matches our numerical integral (via Clenshaw-Curtis quadrature), which is computable in ChebPy via the `sum` command.

In [ ]:
I_ana = 1.2 - np.cos(10) - 0.2 * np.cos(50)
I_num = f.sum()
print(f'analytical : I={I_ana}')
print(f'    ChebPy : I={I_num}')

## Discontinuities

Chebfun is capable of handling certain classes of mathematical nonsmoothness. For example, here we compute the pointwise maximum of two functions, which results in a 'piecewise-smooth' concatenation of twelve individual pieces (in Chebfun & ChebPy terminology this is a collection of 'Funs'). The breakpoints between the pieces (Funs) have been determined by ChebPy in the background by solving the corresponding root-finding problem.

In [ ]:
g = x/5 - 1
h = f.maximum(g)
h

Here's a plot of both `f` and `g`, and their maximum, `h`:

In [ ]:
fig, ax = plt.subplots()
f.plot(ax=ax, linewidth=1, linestyle='--', label='f')
g.plot(ax=ax, linewidth=1, linestyle='--', label='g')
h.plot(ax=ax, label='max(f, g)')
ax.set_ylim([-2.5, 2.5])
ax.legend()
plt.show()

The function `h` is a further Chebfun representation (Chebfun operations such as this are closures) and thus the same set of operations can be applied as normal. Here for instance is the exponential of `h` and its integral:

In [ ]:
np.exp(h).plot()
plt.show()
print(f'integral: {np.exp(h).sum()}')

## Gaussian distribution

Here's a further example, this time related to statistics. We consider the following Chebfun representation of the standardised Gaussian distribution, using a sufficiently wide interval as to facilitate a machine-precision representation. On this occasion we utlilise a slightly different (but still perfectly valid) approach to construction whereby we supply the function handle (in this case, a Python lambda, but more generally any object in possession of a `__call__` attribute) together with the interval of definition.

In [ ]:
def gaussian(x): return 1/np.sqrt(2*np.pi) * np.exp(-.5*x**2)
pdf = chebfun(gaussian, [-15, 15])
ax = pdf.plot()
ax.set_ylim([-.05, .45])
ax.set_title('Standard Gaussian distribution (mean  0, variance 1)')
plt.show()

The integral of any probability density function should be unity, and this is the case for our numerical approximation:

In [ ]:
print(f'integral : {pdf.sum()}')

Suppose we wish to generate quantiles of the distribution. This can be achieved as follows. First we form the cumulative distribution function,
computed as the indefinite integral (`cumsum`) of the density:

In [ ]:
cdf = pdf.cumsum()
ax = cdf.plot()
ax.set_ylim([-0.1, 1.1])
plt.show()

Then it is simply a case of utilising the `roots` command to determine the standardised score (sometimes known as 'z-score') corresponding to the quantile of interest. For example:

In [ ]:
print('quantile    z-score ')
print('--------------------')
for quantile in np.arange(.1, .0, -.01):
    roots = (cdf-quantile).roots()
    print(f'  {quantile*100:2.0f}%       {roots[0]:+5.3f}')

Other distributional properties are also computable. Here's how we can compute the first four normalised and centralised moments (Mean, Variance, Skew, Kurtosis):

In [ ]:
x = pdf.x  # identity on domain of pdf
m1 = (pdf*x).sum()
m2 = (pdf*(x-m1)**2).sum()
m3 = (pdf*(x-m1)**3).sum() / m2**1.5
m4 = (pdf*(x-m1)**4).sum() / m2**2
print('    mean = {:+.4f}'.format(m1))
print('variance = {:+.4f}'.format(m2))
print('    skew = {:+.4f}'.format(m3))
print('kurtosis = {:+.4f}'.format(m4))